## 注意(attention)！开始做前必读项！
所有的代码一定要在这个文件里面编写，不要自己创建一个新的文件
对于提供的数据集，不要改存储地方，也不要修改文件名和内容
不要重新定义函数（如果我们已经定义好的），按照里面的思路来编写。当然，除了我们定义的部分，如有需要可以自行定义函数或者模块
写完之后，重新看一下哪一部分比较慢，然后试图去优化。一个好的习惯是每写一部分就思考这部分代码的时间复杂度和空间复杂度，AI工程是的日常习惯！
这次作业很重要，一定要完成！ 相信会有很多的收获！

In [3]:
import sys
sys.path.append("/usr/local/anaconda2/envs/pt-tf-env/lib/python3.6/site-packages")

In [4]:
import numpy as np
import pandas as pd
from gensim import models
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.externals import joblib
import joblib

加载训练集文件和测试集文件

In [9]:
data_text,data=[],[]
stopWords=[]

def load_data():
    '''
    函数说明：该函数用于加载数据集
    return: 
        -data: 表示所有数据拼接的原始数据
        -data_text: 表示数据集中的特征数据集
        -datatext: 表示经过分词之后的特征数据集
        -stopWords: 表示读取的停用词
    '''
    print('load Pre_process')
    data = pd.concat([
        pd.read_csv('data/train_clean.csv', sep='\t'),
        pd.read_csv('data/dev_clean.csv', sep='\t'),
        pd.read_csv('data/test_clean.csv', sep='\t')
        ])
    print("读取数据集完成")
    data_text = list(data.text)  # .apply(lambda x: x.split(' '))
    datatext = []
    for i in range(len(data_text)):
        datatext.append(data_text[i].split(' '))
    stopWords = open('data/stopwords.txt').readlines()
    print("取停用词完成")
    return data, data_text, datatext, stopWords


In [10]:
data, data_text, datatext, stopWords = load_data()

load Pre_process
读取数据集完成
取停用词完成


In [11]:
data[:3]

,text,label,category_id
0,杏林 芳菲 广东 中医药 曹磊 编著 的 杏林 芳菲 广东 中医药 注重 文化 内涵 的 挖...,文化,6
1,额吉 的 白云 优秀 蒙古文 文学作品 翻译 出版 工程 第五辑 散文 卷 散文集 额吉 的...,文学,0
2,"叶赛宁 抒情诗 选 本书 收入 叶赛宁 的 206 首 抒情诗 , 多为 名篇 佳作 叶赛宁...",文学,0


In [12]:
data_text[:3]

['杏林 芳菲 广东 中医药 曹磊 编著 的 杏林 芳菲 广东 中医药 注重 文化 内涵 的 挖掘 以及 特殊 技艺 的 介绍 , 对于 非 物质 文化遗产 广东 中医药 的 内涵 、 技艺 、 形态 、 历史 演变 、 艺术 价值 等 给予 全面 介绍 深刻 而 直观 地 记录 时代 的 变迁 , 记录 民间 丰富 的 生活 , 图文并茂 , 生动活泼 , 富有 艺术 表现力 , 给 读者 以 文化 审美 的 感受',
 '额吉 的 白云 优秀 蒙古文 文学作品 翻译 出版 工程 第五辑 散文 卷 散文集 额吉 的 白云 共 收入 16 篇 蒙译汉 散文 作品 , 代表 了 建国以来 内蒙古 老中青 三代 蒙古文 散文 创作 的 整体实力 与 成就 这些 散文 作品 抒发 了 作者 对 故乡 故土 、 父辈 亲人 的 思念 热爱 之情 , 也 记录 了 近 三十年 内蒙古 社会 、 文化 、 经济 生活 发生 的 翻天覆地 的 变化 草原 民族 特有 的 浪漫情怀 与 对 草原 生态 现状 的 深切 忧患 跃然纸上 ...',
 '叶赛宁 抒情诗 选 本书 收入 叶赛宁 的 206 首 抒情诗 , 多为 名篇 佳作 叶赛宁 的 诗 从 上 世纪 80 年代 起 在 我国 得到 广泛 的 传播 , 他 的 诗 饱含 醇厚 的 俄罗斯 民族 的 文化底蕴 , 散发 着 俄罗斯 田野 泥土 的 芳香 及 炽烈 的 诗人 情怀 , 洋溢着 意象 艺术 创新 所 带来 的 奇特 艺术 魅力 当代 著名诗人 多里 佐 对 叶赛宁 及其 诗作 给予 了 这样 的 评价 : 他 属于 那些 也许 几百年 才 产生 几个 的 诗人 , 他 不但 进入 了 俄罗斯 文学 , 而且 已经 进入 俄罗斯 的 风景 , 成为 它 不可分割 的 一部分 ...']

In [14]:
datatext[:2]

[['杏林',
  '芳菲',
  '广东',
  '中医药',
  '曹磊',
  '编著',
  '的',
  '杏林',
  '芳菲',
  '广东',
  '中医药',
  '注重',
  '文化',
  '内涵',
  '的',
  '挖掘',
  '以及',
  '特殊',
  '技艺',
  '的',
  '介绍',
  ',',
  '对于',
  '非',
  '物质',
  '文化遗产',
  '广东',
  '中医药',
  '的',
  '内涵',
  '、',
  '技艺',
  '、',
  '形态',
  '、',
  '历史',
  '演变',
  '、',
  '艺术',
  '价值',
  '等',
  '给予',
  '全面',
  '介绍',
  '深刻',
  '而',
  '直观',
  '地',
  '记录',
  '时代',
  '的',
  '变迁',
  ',',
  '记录',
  '民间',
  '丰富',
  '的',
  '生活',
  ',',
  '图文并茂',
  ',',
  '生动活泼',
  ',',
  '富有',
  '艺术',
  '表现力',
  ',',
  '给',
  '读者',
  '以',
  '文化',
  '审美',
  '的',
  '感受'],
 ['额吉',
  '的',
  '白云',
  '优秀',
  '蒙古文',
  '文学作品',
  '翻译',
  '出版',
  '工程',
  '第五辑',
  '散文',
  '卷',
  '散文集',
  '额吉',
  '的',
  '白云',
  '共',
  '收入',
  '16',
  '篇',
  '蒙译汉',
  '散文',
  '作品',
  ',',
  '代表',
  '了',
  '建国以来',
  '内蒙古',
  '老中青',
  '三代',
  '蒙古文',
  '散文',
  '创作',
  '的',
  '整体实力',
  '与',
  '成就',
  '这些',
  '散文',
  '作品',
  '抒发',
  '了',
  '作者',
  '对',
  '故乡',
  '故土',
  '、',
  '父辈',
  '亲人',
  '的',
  '思念',
  '热爱',
  '之

In [15]:
def trainer_tfidf():
    '''
    函数说明：该函数用于训练tfidf的词向量
    return: 
        -tfidf: 表示经过TF-ID模型训练出的词向量
    '''
    print('train tfidf_embedding')
    # 训练tfidf的词向量
    # Convert a collection of raw documents to a matrix of TF-IDF features.
    count_vect = TfidfVectorizer(
        stop_words = stopWords, max_df=0.6, ngram_range=(1, 2))
    tfidf = count_vect.fit(data.text)
    return tfidf

In [16]:
tfidf = trainer_tfidf()

train tfidf_embedding


/usr/local/anaconda2/envs/lyy-pt15-env/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', 'sub', 'sup', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｃ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '一一', '一个', '一些', '一切', '一则', '一方面', '一旦', '一来', '一样', '一番', '一直', '一般', '万一', '上下', '不仅', '不但', '不光', '不单', '不只', '不如', '不怕', '不惟', '不成', '不拘', '不比', '不然', '不特', '不独', '不管', '不论', '不过', '不问', '与其', '与否', '与此同时', '两者', '为了', '为什么', '为何', '为着', '乃至', '之一', '之所以', '之类', '乌乎', '也好', '也就是说', '也罢', '于是', '于是乎', '云云', '人家', '什么', '什么样', '从而', '他人', '他们', '以便', '以免', '以及', '以至', '以至于', '以致', '任何', '任凭',

## 根据数据集训练word2vec的词向量

In [17]:
def trainer_w2v():
    '''
    函数说明：该函数基于Word2vec模型训练词向量
    return: 
        -w2v: 表示经过word2vec模型训练出的词向量
    '''
    print('train word2vec Embedding')
        # 训练w2v的词向量
    w2v = models.Word2Vec(min_count=2,
                                window=3,
                                size=300,
                                sample=6e-5,
                                alpha=0.03,
                                min_alpha=0.0007,
                                negative=15,
                                workers=4,
                                iter=10,
                                max_vocab_size=50000)
        
    w2v.build_vocab(datatext)

    w2v.train(datatext,
                       total_examples=w2v.corpus_count,
                       epochs=15,
                       report_delay=1)
        
    print('train fast_embedding')
    return w2v

In [18]:
w2v = trainer_w2v()

train word2vec Embedding
train fast_embedding


In [19]:
def trainer_fasttext():
    '''
    函数说明：该函数基于FastText模型训练词向量
    return: 
        -fast: 表示经过FastText模型训练出的词向量
    '''
    fast = models.FastText(datatext,
                                size=300,  # 向量维度
                                window=2,  # 移动窗口
                                alpha=0.03,
                                min_count=2,  # 对字典进行截断，小于该数的则会被切掉,增大该值可以减少词表个数
                                iter=10,  # 迭代次数
                                min_n=1,
                                max_n=3,
                                word_ngrams=2,
                                max_vocab_size=50000)
    return fast

In [20]:
fast = trainer_fasttext()

## 保存生成的词向量

In [21]:
def saver():
    '''
    函数说明：该函数存储训练好的模型
    '''
    print('save tfidf model')
    joblib.dump(tfidf, 'tfidf_model')
    print('save word2vec model')
    w2v.save('w2v_model')
    print('save fast model')
    fast.save('fast_model')
    
saver()  

save tfidf model
save word2vec model
save fast model


## 加载模型 

In [22]:
def load():
    '''
    函数说明：该函数加载训练好的模型
    '''
    print('load tfidf_embedding model')
    tfidf = joblib.load('tfidf_model')
    print('load w2v_embedding model')
    w2v = models.KeyedVectors.load('w2v_model')
    print('load fast_embedding model')
    fast = models.FastText.load('fast_model')
    
load()

load tfidf_embedding model
load w2v_embedding model
load fast_embedding model


In [23]:
tfidf.get_feature_names()

['00',
 '00 000',
 '00 03',
 '00 10',
 '00 2014',
 '00 24',
 '00 isbn',
 '00 一代',
 '00 丁当',
 '00 七所',
 '00 三年',
 '00 专家',
 '00 中医',
 '00 中国',
 '00 九年',
 '00 二二',
 '00 二年',
 '00 住校',
 '00 作家',
 '00 作者',
 '00 全国',
 '00 八岁',
 '00 八年',
 '00 出版',
 '00 出版社',
 '00 创作',
 '00 包装',
 '00 古代汉语',
 '00 只为',
 '00 台北',
 '00 名师',
 '00 后们',
 '00 后国漫',
 '00 四年',
 '00 复活节',
 '00 大楼',
 '00 失乐园',
 '00 如何',
 '00 宽容',
 '00 小说',
 '00 年度',
 '00 年轻',
 '00 年轻人',
 '00 年龄层',
 '00 开场',
 '00 开本',
 '00 当当价',
 '00 必读',
 '00 成体',
 '00 抬头',
 '00 教材',
 '00 文化',
 '00 文字',
 '00 新生代',
 '00 新锐',
 '00 晚上',
 '00 根据',
 '00 樱桃',
 '00 武则天',
 '00 每个',
 '00 活泼',
 '00 爱看',
 '00 版次',
 '00 章道',
 '00 第二辑',
 '00 耳朵',
 '00 职场',
 '00 舌尖',
 '00 读者',
 '00 赌石',
 '00 过早地',
 '00 这是',
 '00 这本',
 '00 适应',
 '00 重播',
 '00 阶梯',
 '00 雅克',
 '00 面对',
 '00 页数',
 '00 高兴',
 '00 高考',
 '00 鸡丝',
 '00 黄泳',
 '00 黄泳曲',
 '000',
 '000 00',
 '000 000',
 '000 0000',
 '000 0002',
 '000 2013',
 '000 cctv20',
 '000 one',
 '000 一个',
 '000 一亿',
 '000 一张',
 '000 一版',
 '0

In [27]:
for word in w2v.wv.index2word:
    print(word, w2v[word])
    break

的 [-2.01471835e-01 -2.10209742e-01 -7.01478004e-01  1.60175696e-01
  5.20978391e-01  3.25135529e-01  2.01257765e-01 -3.63359571e-01
  1.07530162e-01  2.59931564e-01  1.24840543e-01 -1.45661021e-02
 -3.08850229e-01 -2.05755159e-01  5.73255420e-01 -5.18662870e-01
  2.59994399e-02 -1.18804678e-01 -3.27116132e-01 -2.76767612e-01
  8.42132047e-03 -3.71708006e-01  3.42649430e-01  4.15655434e-01
  1.15391895e-01 -1.86182126e-01 -4.00007993e-01  3.05900991e-01
 -3.28817099e-01 -3.69022973e-02  2.02856418e-02  2.05725525e-02
 -7.00451553e-01 -7.02408314e-01 -6.77910507e-01 -3.66602987e-01
 -5.16310632e-01 -1.01700999e-01  2.96137214e-01 -1.69101432e-01
 -8.03449564e-03 -4.09607321e-01  1.11723281e-01  2.63527930e-01
 -3.53710473e-01 -2.51021445e-01  1.05488755e-01  1.00822886e-02
 -2.46417850e-01  2.94591486e-01  3.03042918e-01  2.07881719e-01
  1.65381461e-01 -5.73828280e-01 -4.93423343e-02  1.16913378e-01
  2.23264806e-02 -3.17252755e-01 -1.46013781e-01 -2.07169075e-03
 -5.30909300e-01  5.369

/usr/local/anaconda2/envs/lyy-pt15-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [28]:
len( w2v.wv.index2word)

19592